# Shanghai Airport - AirSafe Tracking History usage walkthrough

In this example we will call the AirSafe Tracking History API, plot the results on a map as well as generate some basic statistics from the obtained data.

In [1]:
# Uncomment the lines below when running in the cloud

!mkdir datasets
!wget -O requirements.txt 'https://github.com/spireglobal/aviation/blob/master/jupyter-notebooks/shanghai-airport/requirements.txt?raw=true'
!wget -O datasets/demo.csv 'https://github.com/spireglobal/aviation/blob/master/jupyter-notebooks/shanghai-airport/datasets/demo.csv?raw=true'
!pip install -r requirements.txt

# Uncomment the lines below when running locally, after setting up your local kernel

# %load_ext dotenv
# %dotenv

In [2]:
import getpass

print('If you are in possession of an AirSafe Tracking History API token, please enter it here. If you don\'t, you can just press enter and leave the field blank.')
api_token = getpass.getpass()

If you are in possession of an AirSafe Tracking History API token, please enter it here. If you don	, you can just press enter and leave the field blank.


In [3]:
import os
import plotly.express as px
import pandas as pd
import requests
import json

pd.options.plotting.backend = "plotly"

resp = requests.get(
    "https://api.airsafe.spire.com/v2/targets/history",
    params={
        "start": "2021-03-20T12:00:00Z",
        "end": "2021-03-20T14:30:00Z",
        "latitude_between": "31.02988460440661,31.36374539559339",
        "longitude_between": "121.1632295670742,121.52071043292581",
    },
    headers={"Authorization": f"Bearer {os.getenv('API_TOKEN') if os.getenv('API_TOKEN') else api_token}"},
)


# If we do not have a valid API token, the default pre-downloaded dataset will be used.
if (resp.status_code == 401):
    df = pd.read_csv('datasets/demo.csv')
else:
    data = []
    for line in resp.iter_lines(decode_unicode=True):
        if line and '"target":{' in line:
            data.append(json.loads(line)["target"])
    df = pd.DataFrame(data)

We have now queried the data over Shanghai Hongqiao airport, and generated a Panda DataFrame with the results. We can now easily plot that data on a map using the Plotly Python library.

In [4]:
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', hover_data=['icao_address',"altitude_baro", "on_ground"], zoom=9,
                        mapbox_style="carto-positron", color="icao_address")
fig.show()

## Query a single aircraft and plot its course

Let's now select the aircraft with the most datapoints in the previous query, and run a query for that aircraft only on a longer period of time.

In [5]:
resp = requests.get(
    "https://api.airsafe.spire.com/v2/targets/history",
    params={
        "icao_address": df['icao_address'].value_counts().idxmax(),
        "start": "2021-03-20T12:00:00Z",
        "end": "2021-03-20T20:30:00Z",
    },
    headers={"Authorization": f"Bearer bdBsXdN16oev8QI50yGWAl7pHTmL7PJ7"},
)
data = []
for line in resp.iter_lines(decode_unicode=True):
    if line and '"target":{' in line:
        data.append(json.loads(line)["target"])
df2 = pd.DataFrame(data)
fig = px.scatter_mapbox(df2, lat='latitude', lon='longitude', hover_data=['timestamp', 'icao_address',"altitude_baro", "on_ground"],
                        zoom=4,
                        mapbox_style="carto-positron")
fig.show()

## Plot the aircraft altitude

Let's now see analyse the altitude variations of the flight we previously requested.

We will just plot the `altitude_baro` field in a line graph to do so.

Note that we have also generated a second line in the graph that indiciates the difference in altitude between each data point. This will help noticing potential jumps in the data, if the difference is too high.

In [6]:
from plotly.subplots import make_subplots

fig = px.line(df2, x="timestamp", y="altitude_baro")
fig.add_scatter(y=df2['altitude_baro'].diff().abs(), x=df2['timestamp'],  mode='lines', name="alt_diff")
fig.show()

## Generate aircraft type statistics

Let's now generate some statistics regarding the types of aircraft present in the zone we queried.
We can first group the entries by `aircraft_type_icao`, then select the `icao_address` column which we will then `nunique()` to count the number of unique ICAO addresses present for each aircraft type. 
We can then simply plot it!

In [7]:
unique_aircraft_type = df.groupby('aircraft_type_icao')['icao_address'].nunique()
plot = unique_aircraft_type.plot(kind="bar", y='icao_address', text='icao_address')
plot.update_traces(textposition='outside')

We can also find out the different destinations deserved by the captured flights during this period. We can group our data points by `arrival_airport_icao` and `airline_name` and `nunique()` the results so we get the exact number of points for this combination.

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)
test = df.groupby(['arrival_airport_icao', 'airline_name']).nunique().sort_values(by=['arrival_airport_icao', 'flight_number'], ascending=False)
test['flight_number'].to_frame()